In [22]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os, re, sys
from sklearn.model_selection import train_test_split
import skimage.io
import gc

In [23]:
# test if GPUs are available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# set project root, maybe you need to firstly 
# add shortcut of CS 766 Project to drive.
project_root = './drive/MyDrive/CS 766 Project/Project Coding and Data Files'

In [26]:
# class to initialize CNNs
class OriginCNN(object):
  """docstring for OriginCNN"""
  def __init__(self):
    self.optimizer = 'adam'
    # self.optimizer = tf.keras.optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
    # self.loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.loss = 'categorical_crossentropy'
    # will add more properties


  def build(self):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu', input_shape=(1000, 1000, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), strides=(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))

    model.summary()
    model.compile(optimizer=self.optimizer,
                loss=self.loss,
                metrics=['accuracy'])
    self.model = model


  def train(self, train_images, train_labels, epochs):
    self.history = self.model.fit(train_images, train_labels, epochs=epochs, validation_split=0.1)


  def evaluate(self, test_images, test_labels):
    _, test_acc = self.model.evaluate(test_images, test_labels, verbose=2)
    return test_images.shape[0], test_acc

  def save_model(self, filepath):
    self.model.save(filepath)

  def load_model(self, filepath):
    self.model = tf.keras.models.load_model(filepath)

  def extract_feature(self, images):
    extract = models.Model(self.model.inputs, self.model.layers[-3].output)
    return extract.predict(images)

In [27]:
class DR_resized(object):
  """docstring for DR_resized"""
  def __init__(self, is_training, batch_id):
    if(is_training):
      # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Even_Class_Distribution_Datasets", "Processed_Xtrain_Batch%d_even.npy" % batch_id))
      # all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Stratified_Random_Sampling_Datasets", "Processed_Xtrain_Batch%d.npy" % batch_id))
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Even Class Distribution Datasets", "Ytrain", "Ytrain_Batch%d_even.npy" % batch_id), allow_pickle=True)
      # all_labels = np.load(os.path.join(project_root, "Data Batches", "Stratified Random Sampling Datasets", "Ytrain", "Ytrain_Batch%d.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])
      
      # one hot
      all_labels_one_hot = np.zeros((all_labels.size, 5))
      all_labels_one_hot[np.arange(all_labels.size),all_labels] = 1

      # without one hot
      # all_labels = np.expand_dims(all_labels, axis=-1)

      self.train_images = all_images
      self.train_labels = all_labels_one_hot
    
    else:
        # load images
      all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Even_Class_Distribution_Datasets", "Processed_Xest_Batch%d_even.npy") % batch_id)
      # all_images = np.load(os.path.join(project_root, "Processed_Data_Batches", "Stratified_Random_Sampling_Datasets", "Processed_Xtest_Batch%d.npy") % batch_id)
      # add a dimension for channels
      all_images = np.expand_dims(all_images, axis=-1)

      # load labels
      all_labels = np.load(os.path.join(project_root, "Data Batches", "Even Class Distribution Datasets", "Ytest", "Yest_Batch%d_even.npy" % batch_id), allow_pickle=True)
      # all_labels = np.load(os.path.join(project_root, "Data Batches", "Stratified Random Sampling Datasets", "Ytest", "Ytest_Batch%d.npy" % batch_id), allow_pickle=True)
      all_labels = np.array([item[1] for item in all_labels])

      # one hot
      all_labels_one_hot = np.zeros((all_labels.size, 5))
      all_labels_one_hot[np.arange(all_labels.size),all_labels] = 1

      # without one hot
      # all_labels = np.expand_dims(all_labels, axis=-1)
      
      self.test_images = all_images
      self.test_labels = all_labels_one_hot
  
  def clear(self):
    self.train_images = None
    self.validate_images = None
    self.test_images = None
    self.train_labels = None
    self.validate_labels = None
    self.test_labels = None


In [31]:
# initialize CNN
myModel = OriginCNN()
myModel.build()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 333, 333, 32)      320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 166, 166, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)              

In [32]:
# training process
file_num = 9
epoch_num = 5
for i in range(file_num):
  myData = DR_resized(True, i)
  print("data batch %d loaded" % i)
  myModel.train(myData.train_images, myData.train_labels, epoch_num)
  print("data batch %d trained" % i)
  myData.clear()
  gc.collect()

data batch 0 loaded
Epoch 1/5
9/9 [==============================] - 1s 99ms/step - loss: 1.6159 - accuracy: 0.1893 - val_loss: 2.1453 - val_accuracy: 0.0000e+00
Epoch 2/5
9/9 [==============================] - 1s 73ms/step - loss: 1.5712 - accuracy: 0.2239 - val_loss: 2.1298 - val_accuracy: 0.0000e+00
Epoch 3/5
9/9 [==============================] - 1s 78ms/step - loss: 1.5738 - accuracy: 0.2483 - val_loss: 2.0299 - val_accuracy: 0.0000e+00
Epoch 4/5
9/9 [==============================] - 1s 79ms/step - loss: 1.5570 - accuracy: 0.2830 - val_loss: 2.1751 - val_accuracy: 0.0000e+00
Epoch 5/5
9/9 [==============================] - 1s 72ms/step - loss: 1.5375 - accuracy: 0.3596 - val_loss: 2.2698 - val_accuracy: 0.0000e+00
data batch 0 trained
data batch 1 loaded
Epoch 1/5
9/9 [==============================] - 1s 82ms/step - loss: 1.5834 - accuracy: 0.2222 - val_loss: 1.9909 - val_accuracy: 0.0000e+00
Epoch 2/5
9/9 [==============================] - 1s 81ms/step - loss: 1.5780 - accuracy

In [33]:
# testing process
file_num = 2
num_list = []
acc_list = []
for i in range(file_num):
  myData = DR_resized(False, i)
  num, acc = myModel.evaluate(myData.test_images, myData.test_labels)
  print(num, acc)
  num_list.append(num)
  acc_list.append(acc)
  myData.clear()
  gc.collect()

10/10 - 0s - loss: 1.6752 - accuracy: 0.1968
310 0.1967741996049881
14/14 - 1s - loss: 1.6749 - accuracy: 0.2341
440 0.2340909093618393


In [19]:
# compute overall accuracy
num_list = np.array(num_list)
acc_list = np.array(acc_list)
acc_overall = (num_list * acc_list).sum() / num_list.sum()
print(acc_overall)

0.733040004587972


In [ ]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y %H:%M:%S")
print("date and time =", dt_string)
myModel.save_model(os.path.join(project_root, "Trained Models", dt_string))

date and time = 16-03-2021 01:13:01
INFO:tensorflow:Assets written to: ./drive/MyDrive/CS 766 Project/Project Coding and Data Files/Trained Models/16-03-2021 01:13:01/assets


In [ ]:
# this part is for loading parameters and extracting features
# load parameters
parameter_version = "16-03-2021 01:13:01"
if not os.path.exists(os.path.join(project_root, "features", parameter_version)):
    os.mkdir(os.path.join(project_root, "features", parameter_version))

my_model_pretrained = OriginCNN()
my_model_pretrained.build()
my_model_pretrained.load_model(os.path.join(project_root, "Trained Models", parameter_version))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 333, 333, 32)      320       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 166, 166, 32)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 55, 55, 32)        9248      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 27, 27, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 9, 9, 32)          9248      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 512)             

In [21]:
# load training data, extract features and save to npy files
file_num = 9
for i in range(file_num):
  myData = DR_resized(True, i)
  feature = my_model_pretrained.extract_feature(myData.train_images)
  # feature = myModel.extract_feature(myData.train_images)
  np.save(os.path.join(project_root, "features", parameter_version, "Xtrain_feature%d.npy" % i), feature)
  print("Xtrain_feature%d.npy" % i)
  print(feature)
  myData.clear()
  gc.collect()

# load testing data, extract features and save to npy files
file_num = 2
for i in range(file_num):
  myData = DR_resized(False, i)
  feature = my_model_pretrained.extract_feature(myData.test_images)
  np.save(os.path.join(project_root, "features", parameter_version, "Xtest_feature%d.npy" % i), feature)
  print("Xtest_feature%d.npy" % i)
  myData.clear()
  gc.collect()

Xtrain_feature0.npy
[[0.13327868 0.20634948 0.90001553 ... 0.         0.         0.21805297]
 [0.01951008 0.37221256 0.7282312  ... 0.         0.         0.10665232]
 [0.17231773 0.41383412 0.7655909  ... 0.         0.         0.10913053]
 ...
 [0.02165465 0.37796298 0.7282312  ... 0.         0.         0.        ]
 [0.10473075 0.41896763 0.7282312  ... 0.         0.         0.02509846]
 [0.14702679 0.5428476  0.79617906 ... 0.         0.         0.08386298]]
Xtrain_feature1.npy
[[0.03505823 0.45131296 0.8865059  ... 0.         0.         0.        ]
 [0.02727759 0.3653634  0.76590776 ... 0.         0.         0.18827017]
 [0.15299256 0.4470656  0.804988   ... 0.         0.         0.24860455]
 ...
 [0.02606652 0.46200976 0.8601537  ... 0.         0.         0.07274511]
 [0.02449954 0.4146142  0.7317382  ... 0.         0.         0.07876874]
 [0.1174073  0.3688428  0.89758325 ... 0.         0.         0.25072953]]
Xtrain_feature2.npy
[[0.04790842 0.37480325 0.8757356  ... 0.         0.

NameError: ignored

In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models

In [10]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")
print(y_train.shape)
print(y_train)


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
print(y_train)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(60000,)
[5 0 4 ... 5 6 8]
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
